In [1]:
import os
os.cpu_count()

256

In [2]:
import os
os.cpu_count()

256

In [3]:
import psutil

In [4]:
psutil.cpu_count(logical=False)

128

In [5]:
psutil.cpu_count(logical=True)

256

In [12]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [13]:
# Load the GSM dataset with the 'main' configuration (or 'socratic')
dataset = load_dataset("gsm8k", "main", split="test[:1%]")  # 1% subset for faster testing


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import matplotlib.pyplot as plt
from datasets import load_dataset

# Function to load the model and tokenizer (called only once)
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.to('cpu')
    return model, tokenizer

# Function to test latency and get output
def test_latency_and_output(model, tokenizer, input_text, num_cpus, batch_size=1):
    # Set the number of threads to use for CPU-based inference
    torch.set_num_threads(num_cpus)
    
    inputs = tokenizer(input_text, return_tensors="pt")

    # Warm-up the model (first few runs may be slow)
    for _ in range(3):
        model.generate(inputs['input_ids'], max_new_tokens=50)  # Using max_new_tokens instead of max_length

    # Measure latency and capture output
    start_time = time.time()
    generated_outputs = []
    for _ in range(batch_size):
        # Use max_new_tokens to generate tokens after the input sequence
        output = model.generate(inputs['input_ids'], max_new_tokens=50)  # Generate up to 50 new tokens
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_outputs.append(generated_text)
    end_time = time.time()

    latency = (end_time - start_time) / batch_size  # average latency per inference
    return latency, generated_outputs

# Load the GSM dataset with the 'main' configuration (or 'socratic')
dataset = load_dataset("gsm8k", "main", split="test[:1%]")  # 1% subset for faster testing
# Load the full test split
#dataset = load_dataset("gsm8k", "main", split="test")  # Full test set
# Load the first 100 samples from the test split
#dataset = load_dataset("gsm8k", "main", split="test[:100]")  # First 100 samples



# Test latency and output for different CPU configurations
cpu_configs = [1,2,3,4]  # List of CPU configurations to test (1, 4, 8 CPUs)
model_name = 'gpt2'  # Replace with your model name

# Load the model and tokenizer once
model, tokenizer = load_model(model_name)

# Store results
latencies = []
generated_answers = []

# Run the tests for each CPU config
for cpus in cpu_configs:
    latency = 0
    generated_output = []
    
    # Measure latency on a few samples from the GSM dataset
    for example in dataset:
        input_text = example['question']  # Using the 'question' field now
        
        # Test latency and capture output
        single_latency, output = test_latency_and_output(model, tokenizer, input_text, cpus)
        latency += single_latency
        generated_output.append(output[0])  # Only take the first generated output
    
    # Average latency for the current CPU configuration
    avg_latency = latency / len(dataset)
    latencies.append(avg_latency)
    print(f"\nAverage Latency with {cpus} CPUs: {avg_latency:.4f} seconds per inference")
    print(f"Generated output with {cpus} CPUs: {generated_output[0]}")  # Displaying the first output for inspection

# Optionally plot the results
plt.plot(cpu_configs, latencies, marker='o')
plt.xlabel('Number of CPUs')
plt.ylabel('Latency (seconds)')
plt.title(f'Latency vs. Number of CPUs for {model_name} on GSM dataset')
plt.grid(True)
plt.show()


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Average Latency with 1 CPUs: 2.0903 seconds per inference
Generated output with 1 CPUs: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

‪

‪

‪

‪

‪

‪

‪

‪

‪

‪

‪

‪




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [15]:
from datasets import load_dataset

# Load the GSM8K dataset (first 1% of the test set for faster processing)
dataset = load_dataset("gsm8k", "main", split="test[:1%]")

# Print the column names to see the available fields
print(f"Dataset columns: {dataset.column_names}")

# Print the first example to check its structure
print(f"First example in the dataset: {dataset[0]}")


Dataset columns: ['question', 'answer']
First example in the dataset: {'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}


In [18]:
print(f"Number of examples in test split: {len(dataset)}")

Number of examples in test split: 13


In [19]:
print(f"First example in the dataset: {dataset}")

First example in the dataset: Dataset({
    features: ['question', 'answer'],
    num_rows: 13
})


In [20]:
print(f"First example in the dataset: {dataset[4]}")

First example in the dataset: {'question': "Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?", 'answer': 'If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.\nIf she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-15-25=<<60-15-25=20>>20 cups of chicken feed.\n#### 20'}


In [21]:
dataset = load_dataset("gsm8k", "main", split="test")

In [22]:
print(f"First example in the dataset: {dataset}")

First example in the dataset: Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})
